In [1]:
import numpy as np 
import pandas as pd 


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd

def load_data(path,batch_size = 32):
    df = pd.read_csv(path)
    df.columns = ['input_word','target_word']
    
    # Define maximum sequence lengths for letters
    max_input_len = max(len(word) for word in df['input_word'])
    max_target_len = max(len(word) for word in df['target_word'])

    # Define vocabulary mappings for letters
    input_letter_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2}  # Add special tokens
    target_letter_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2}  # Add special tokens
    letter_idx = 3

    input_str = ''
    target_str = ''
    # Preprocess the data and update vocabulary mappings for letters
    for input_word, target_word in zip(df['input_word'], df['target_word']):
        input_str += input_word
        target_str += target_word


    # Update vocabulary mappings for input letters
    for letter in sorted(set(input_str)):
        input_letter_vocab[letter] = letter_idx
        letter_idx += 1
    letter_idx = 3
    # Update vocabulary mappings for target letters
    for letter in sorted(set(target_str)):
        if letter not in target_letter_vocab:
            target_letter_vocab[letter] = letter_idx
            letter_idx += 1

    # Tokenize function at the letter level
    def tokenize_input_letters(word, vocab, max_len):
        token_ids = [vocab[char] for char in word if char in vocab]
        padded = token_ids[:max_len] + [vocab['<pad>']] * (max_len - len(token_ids))
        return torch.tensor(padded)

    def tokenize_target_letters(word, vocab, max_len):
        token_ids = [vocab[char] for char in word if char in vocab]
        padded =  [vocab['<pad>']]+ token_ids[:max_len] +[vocab['<pad>']] * (max_len - len(token_ids))
        return torch.tensor(padded)
    

    # Custom Dataset class for letter-level tokenization
    class CustomDataset(Dataset):
        def __init__(self, input_data, target_data, input_vocab, target_vocab, max_input_len, max_target_len):
            self.input_data = input_data
            self.target_data = target_data
            self.input_vocab = input_vocab
            self.target_vocab = target_vocab
            self.max_input_len = max_input_len
            self.max_target_len = max_target_len

        def __len__(self):
            return len(self.input_data)

        def __getitem__(self, idx):
            input_word = self.input_data[idx]
            target_word = self.target_data[idx]

            # Tokenize input and target words at the letter level
            input_letters = tokenize_input_letters(input_word, self.input_vocab, self.max_input_len)
            target_letters = tokenize_target_letters(target_word, self.target_vocab, self.max_target_len)

            return input_letters, target_letters

    # Create DataLoader
    custom_dataset = CustomDataset(df['input_word'], df['target_word'], input_letter_vocab, target_letter_vocab, max_input_len, max_target_len)
    data_loader1 = DataLoader(custom_dataset, batch_size=batch_size, shuffle = False )
    
    return custom_dataset,data_loader1, input_letter_vocab, target_letter_vocab, max_input_len, max_target_len



In [3]:
path1 = '/kaggle/input/aksharantar/aksharantar_sampled/ben/ben_train.csv'
custom_dataset1,train_loader_ben,a,b,_,_ = load_data(path1,batch_size = 64)
path2 = '/kaggle/input/aksharantar/aksharantar_sampled/ben/ben_valid.csv'
custom_dataset,val_loader_ben,_,_,_,_ = load_data(path2,batch_size = 64)
print(a,b)

{'<pad>': 0, '<sos>': 1, '<eos>': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28} {'<pad>': 0, '<sos>': 1, '<eos>': 2, 'ঁ': 3, 'ং': 4, 'ঃ': 5, 'অ': 6, 'আ': 7, 'ই': 8, 'ঈ': 9, 'উ': 10, 'ঊ': 11, 'ঋ': 12, 'এ': 13, 'ঐ': 14, 'ও': 15, 'ঔ': 16, 'ক': 17, 'খ': 18, 'গ': 19, 'ঘ': 20, 'ঙ': 21, 'চ': 22, 'ছ': 23, 'জ': 24, 'ঝ': 25, 'ঞ': 26, 'ট': 27, 'ঠ': 28, 'ড': 29, 'ঢ': 30, 'ণ': 31, 'ত': 32, 'থ': 33, 'দ': 34, 'ধ': 35, 'ন': 36, 'প': 37, 'ফ': 38, 'ব': 39, 'ভ': 40, 'ম': 41, 'য': 42, 'র': 43, 'ল': 44, 'শ': 45, 'ষ': 46, 'স': 47, 'হ': 48, '়': 49, 'া': 50, 'ি': 51, 'ী': 52, 'ু': 53, 'ূ': 54, 'ৃ': 55, 'ে': 56, 'ৈ': 57, 'ো': 58, 'ৌ': 59, '্': 60, 'ৎ': 61, '২': 62}


# **Seq2Seq Bidirectional Model**

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm


# Encoder class
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, embed_size, encoder_layers=1, drop_prob=0.5, cell_type='gru', bidirectional=False):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.encoder_layers = encoder_layers
        self.cell_type = cell_type
        self.bidirectional = bidirectional
        self.dropout = nn.Dropout(drop_prob)
        self.embedding = nn.Embedding(input_size, embed_size)
        
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(embed_size, hidden_size, encoder_layers, dropout=drop_prob, bidirectional=bidirectional, batch_first=True)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(embed_size, hidden_size, encoder_layers, dropout=drop_prob, bidirectional=bidirectional, batch_first=True)
        else:
            self.rnn = nn.RNN(embed_size, hidden_size, encoder_layers, dropout=drop_prob, bidirectional=bidirectional, batch_first=True)
    
    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, hidden = self.rnn(embedded)
        
        if self.cell_type == 'lstm':
            hidden_states, cell_states = hidden
            if self.bidirectional:
                return torch.sum(hidden_states[:-2],dim= 0, keepdim = True), torch.sum(cell_states[-2:],dim= 0, keepdim = True)
            else:
                return hidden_states[-1].unsqueeze(0), cell_states[-1].unsqueeze(0)
        else:
            if self.bidirectional:
                return torch.sum(hidden[:-2],dim= 0, keepdim = True)
            else:
                return hidden[-1].unsqueeze(0)
        
# Decoder class
class Decoder(nn.Module):
    def __init__(self, hidden_size, embed_size, output_size, decoder_layers=1, drop_prob=0.5, cell_type='gru'):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.decoder_layers = decoder_layers
        self.cell_type = cell_type
        self.dropout = nn.Dropout(drop_prob)
        self.embedding = nn.Embedding(output_size, embed_size)
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(embed_size, hidden_size, decoder_layers, dropout=drop_prob, batch_first=True)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(embed_size, hidden_size, decoder_layers, dropout=drop_prob, batch_first=True)
        else:
            self.rnn = nn.RNN(embed_size, hidden_size, decoder_layers, dropout=drop_prob, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden):
        x = x.unsqueeze(1)
        embedded = self.dropout(self.embedding(x))
        output, hidden = self.rnn(embedded, hidden)
        output = self.fc(output)
        return output.squeeze(1), hidden

# Sequence to sequence class
class Seq2Seq(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, embed_size, encoder_layers=1, decoder_layers=1, drop_prob=0.3, cell_type='gru', bidirectional= True):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, embed_size, encoder_layers, drop_prob, cell_type, bidirectional)
        self.decoder = Decoder(hidden_size, embed_size, output_size, decoder_layers, drop_prob, cell_type)

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.size(0)
        target_len = target.size(1)
        output_vocab_size = output_size #self.decoder.embedding.num_embeddings

        outputs = torch.zeros(batch_size, target_len, output_vocab_size).to(source.device)

        encoder_hidden = self.encoder(source)  # , encoder_cell
        decoder_hidden = self._init_decoder_hidden(encoder_hidden)  # Initialize decoder hidden state
        decoder_input = target[:, 0]  # start token

        for t in range(1, target_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[:, t] = decoder_output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            t1 = decoder_output.argmax(1)
            decoder_input = target[:, t] if teacher_force else t1

        return outputs

    def _init_decoder_hidden(self, encoder_hidden):
        decoder_layers = self.decoder.decoder_layers
        encoder_layers = encoder_hidden[0].shape[0] if isinstance(encoder_hidden, tuple) else encoder_hidden.shape[0]

        if self.decoder.cell_type == 'lstm':
            if encoder_layers < decoder_layers:
                # Pad the encoder hidden state with zeros to match decoder_layers
                encoder_hidden = (
                    torch.cat(
                        [encoder_hidden[0], torch.zeros(decoder_layers - encoder_layers, *encoder_hidden[0].shape[1:], device=encoder_hidden[0].device)],
                        dim=0),torch.cat(
                        [encoder_hidden[1], torch.zeros(decoder_layers - encoder_layers, *encoder_hidden[1].shape[1:], device=encoder_hidden[1].device)],
                        dim=0))
            if encoder_hidden[0].shape[0] != decoder_layers:
                # If encoder layers and decoder layers are different, adjust the hidden state
                encoder_hidden = (encoder_hidden[0][:decoder_layers], encoder_hidden[1][:decoder_layers])

        else:
            if encoder_layers < decoder_layers:
                # Pad the encoder hidden state with zeros to match decoder_layers
                encoder_hidden = torch.cat(
                    [encoder_hidden, torch.zeros(decoder_layers - encoder_layers, *encoder_hidden.shape[1:], device=encoder_hidden.device)],
                    dim=0
                )
            if encoder_hidden.shape[0] != decoder_layers:
                # If encoder layers and decoder layers are different, adjust the hidden state
                encoder_hidden = encoder_hidden[:decoder_layers]

        return encoder_hidden


# **Train and evaluate**

In [5]:
    
# Training function
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for latin, devanagari in dataloader:  
        latin = latin.to(device)
        devanagari = devanagari.to(device)
        
        optimizer.zero_grad()
        
        output = model(latin, devanagari)
        output_dim = output.shape[-1]
        output = output.view(-1, output_dim)
        devanagari = devanagari.view(-1)
        
        loss = criterion(output, devanagari)
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    return model, total_loss / len(dataloader)

# Evaluation function
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for latin, devanagari in dataloader:  #tqdm(dataloader, desc='Evaluating', unit='batch'):
            latin = latin.to(device)
            devanagari = devanagari.to(device)
            
            output = model(latin, devanagari,teacher_forcing_ratio=0.0)
                                           
            output_dim = output.shape[-1]                                                   
           
            loss = criterion(output.view(-1, output_dim), devanagari.view(-1))
            total_loss += loss.item()
            
            max_values ,max_index = torch.max(output, 2) #output.argmax(dim=1)
            ind = max_index > 9
            max_index[ind] -= 2
#             print(f"prediction:{max_index} actual:{devanagari}")
            correct1=(max_index == devanagari).all(dim=1).sum().item()  # Calculate word accuracy
#             
            total_correct += correct1
            total_samples += devanagari.size(0)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_samples
    
    return avg_loss, accuracy*100

# Example usage
input_size = 30  # Number of Latin characters
output_size = 70  # Number of Devanagari characters
embed_size = 128
hidden_size = 128
encoder_layers = 3
decoder_layers = 2
cell_type = 'lstm'
batch_size = 64
num_epochs = 20
drop_prob = 0.2
learning_rate = 0.001
bidirectional = True


# Initialize the model, criterion, and optimizer
model = Seq2Seq(input_size, output_size, hidden_size,embed_size, encoder_layers,decoder_layers,drop_prob, cell_type,bidirectional)
print(model)

# model = Attention_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
# for epoch in range(num_epochs):
#     trained_model, train_loss = train(model, train_loader_ben, criterion, optimizer, device)
#     val_loss, val_accuracy = evaluate(trained_model, val_loader_ben, criterion, device)
#     model = trained_model
#     print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(30, 128)
    (rnn): LSTM(128, 128, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(70, 128)
    (rnn): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=128, out_features=70, bias=True)
  )
)


# **Wandb Setup**

In [5]:
# !pip install wandb
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [6]:
wandb.login(key='bb3c7761be2856a8335d16d1483149380482ae9e')#bb3c7761be2856a8335d16d1483149380482ae9e


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'embedding_size':{
            'values': [16,32,64,128,256]
        },
        'dropout': {
            'values': [0.3, 0.2,0.5]
        },
        'encoder_layers': {
            'values': [1,2,3]
        },
        'decoder_layers':{
            'values': [1,2,3]
        },
        'hidden_layer_size':{
            'values': [16,32,64,128,256]
        },
        'cell_type': {
            'values': [ 'lstm', 'rnn', 'gru']
        },
        'bidirectional': {
            'values': [True, False]
        },
        'batch_size': {
            'values': [32,64]
        },
        'num_epochs': {
            'values': [10,12]
        },
        'learning_rate': {
            'values': [0.01,0.001]
        }
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='DL_assignment_3')


Create sweep with ID: 9hpnde6s
Sweep URL: https://wandb.ai/abanisingha1997/DL_assignment_3/sweeps/9hpnde6s


In [8]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''

    with wandb.init() as run:
        run_name="ct-"+str(wandb.config.cell_type)+"_el-"+str(wandb.config.encoder_layers)+"_dl-"+str(wandb.config.decoder_layers)+"_drop-"+str(wandb.config.dropout)+"_es-"+str(wandb.config.embedding_size)+"_hs-"+str(wandb.config.hidden_layer_size)+"_bs-"+str(wandb.config.batch_size)+"_ep-"+str(wandb.config.num_epochs)+"lr"+str(wandb.config.learning_rate)
        wandb.run.name=run_name

        
        model = Seq2Seq(input_size=30, output_size=70, hidden_size=wandb.config.hidden_layer_size,embed_size=wandb.config.embedding_size,encoder_layers=wandb.config.encoder_layers,
                        decoder_layers=wandb.config.decoder_layers,drop_prob=wandb.config.dropout, cell_type=wandb.config.cell_type, bidirectional=wandb.config.bidirectional)
        print(model)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        path1 = '/kaggle/input/aksharantar/aksharantar_sampled/ben/ben_train.csv'
        custom_dataset1,train_loader_ben,a,b,_,_ = load_data(path1,batch_size = wandb.config.batch_size)
        path2 = '/kaggle/input/aksharantar/aksharantar_sampled/ben/ben_valid.csv'
        custom_dataset,val_loader_ben,_,_,_,_ = load_data(path2,batch_size = wandb.config.batch_size)

        # Training loop
        for epoch in range(wandb.config.num_epochs):
            trained_model, train_loss = train(model, train_loader_ben, criterion, optimizer, device)
            val_loss, val_accuracy = evaluate(trained_model, val_loader_ben, criterion, device)
            model = trained_model
            wandb.log({'Epoch': epoch, 'train_loss': train_loss , ' val_loss': val_loss, 'val_accuracy':val_accuracy})
            print(f'Epoch {epoch+1}/{wandb.config.num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        
wandb.agent(sweep_id, function= main,count= 30) # calls main function for count number of times.
wandb.finish()

wandb: Agent Starting Run: c3fl6c58 with config:
wandb: 	batch_size: 64
wandb: 	bidirectional: True
wandb: 	cell_type: lstm
wandb: 	decoder_layers: 1
wandb: 	dropout: 0.5
wandb: 	embedding_size: 64
wandb: 	encoder_layers: 2
wandb: 	hidden_layer_size: 64
wandb: 	learning_rate: 0.01
wandb: 	num_epochs: 12
wandb: Currently logged in as: abanisingha1997. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(30, 64)
    (rnn): LSTM(64, 64, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(70, 64)
    (rnn): LSTM(64, 64, batch_first=True, dropout=0.5)
    (fc): Linear(in_features=64, out_features=70, bias=True)
  )
)
Epoch 1/12, Train Loss: 1.0375, Val Loss: 2.5188, Val Accuracy: 0.8059
Epoch 2/12, Train Loss: 0.8383, Val Loss: 2.7877, Val Accuracy: 1.7827
Epoch 3/12, Train Loss: 0.7780, Val Loss: 2.9903, Val Accuracy: 3.4432
Epoch 4/12, Train Loss: 0.7457, Val Loss: 3.0975, Val Accuracy: 4.0293
Epoch 5/12, Train Loss: 0.7274, Val Loss: 3.1876, Val Accuracy: 4.7375
Epoch 6/12, Train Loss: 0.7164, Val Loss: 3.2424, Val Accuracy: 4.4444
Epoch 7/12, Train Loss: 0.7074, Val Loss: 3.2720, Val Accuracy: 4.9573
Epoch 8/12, Train Loss: 0.6893, Val Loss: 3.3848, Val Accuracy: 5.1526
Epoc

val_loss,▁▃▅▅▆▇▇█████
Epoch,▁▂▂▃▄▄▅▅▆▇▇█
train_loss,█▄▃▂▂▂▂▁▁▁▁▁
val_accuracy,▁▂▄▅▅▅▆▆▇▇▇█
val_loss,3.4324
Epoch,11
train_loss,0.66855
val_accuracy,6.95971


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: njhqmcd4 with config:
wandb: 	batch_size: 64
wandb: 	bidirectional: True
wandb: 	cell_type: lstm
wandb: 	decoder_layers: 3
wandb: 	dropout: 0.3
wandb: 	embedding_size: 64
wandb: 	encoder_layers: 3
wandb: 	hidden_layer_size: 16
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 10


...

val_loss,▁▃▄▅▅▆▆▇▇▇██
Epoch,▁▂▂▃▄▄▅▅▆▇▇█
train_loss,█▄▃▂▂▂▂▂▁▁▁▁
val_accuracy,▁▄▅▆▆▇▇▇████
val_loss,4.66313
Epoch,11
train_loss,0.33721
val_accuracy,28.57143


wandb: Agent Starting Run: 0i106cmx with config:
wandb: 	batch_size: 64
wandb: 	bidirectional: False
wandb: 	cell_type: gru
wandb: 	decoder_layers: 1
wandb: 	dropout: 0.3
wandb: 	embedding_size: 128
wandb: 	encoder_layers: 3
wandb: 	hidden_layer_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 12


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(30, 128)
    (rnn): GRU(128, 256, num_layers=3, batch_first=True, dropout=0.3)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(70, 128)
    (rnn): GRU(128, 256, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=256, out_features=70, bias=True)
  )
)
Epoch 1/12, Train Loss: 1.0431, Val Loss: 2.6392, Val Accuracy: 1.4164
Epoch 2/12, Train Loss: 0.6819, Val Loss: 3.0230, Val Accuracy: 6.8620
Epoch 3/12, Train Loss: 0.5701, Val Loss: 3.3008, Val Accuracy: 12.1123
Epoch 4/12, Train Loss: 0.5111, Val Loss: 3.5039, Val Accuracy: 16.6056
Epoch 5/12, Train Loss: 0.4780, Val Loss: 3.6454, Val Accuracy: 18.6325
Epoch 6/12, Train Loss: 0.4503, Val Loss: 3.7440, Val Accuracy: 19.8291
Epoch 7/12, Train Loss: 0.4315, Val Loss: 3.8830, Val Accuracy: 20.8303
Epoch 8/12, Train Loss: 0.4230, Val Loss: 3.8875, Val Accuracy: 22.3687
Epoch 9/12, T

val_loss,▁▃▄▅▆▆▇▇▇███
Epoch,▁▂▂▃▄▄▅▅▆▇▇█
train_loss,█▄▃▂▂▂▂▁▁▁▁▁
val_accuracy,▁▃▄▆▆▇▇▇████
val_loss,4.12688
Epoch,11
train_loss,0.37914
val_accuracy,24.61538


# **Best model**

In [15]:
# Best hyperparameter configuration
input_size = 30  # Number of Latin characters
output_size = 70  # Number of Devanagari characters
embed_size = 128
hidden_size = 256
encoder_layers = 3
decoder_layers = 3
cell_type = 'lstm'
batch_size = 64
num_epochs = 11
drop_prob = 0.2
learning_rate = 0.001
bidirectional = True

# Assuming you have loaded your dataset into train_loader and val_loader

# Initialize the model, criterion, and optimizer
Best_model = Seq2Seq(input_size, output_size, hidden_size,embed_size, encoder_layers,decoder_layers,drop_prob, cell_type,bidirectional)
print(Best_model)

# model = Attention_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Best_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Best_model.parameters(), lr=learning_rate)

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(30, 128)
    (rnn): LSTM(128, 256, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(70, 128)
    (rnn): LSTM(128, 256, num_layers=3, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=256, out_features=70, bias=True)
  )
)


In [16]:
# Training loop
for epoch in range(num_epochs):
    trained_model, train_loss = train(Best_model, train_loader_ben, criterion, optimizer, device)
    model = trained_model

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}')
    

Epoch 1/11, Train Loss: 1.2733
Epoch 2/11, Train Loss: 0.9604
Epoch 3/11, Train Loss: 0.6558
Epoch 4/11, Train Loss: 0.5229
Epoch 5/11, Train Loss: 0.4556
Epoch 6/11, Train Loss: 0.4190
Epoch 7/11, Train Loss: 0.3916
Epoch 8/11, Train Loss: 0.3743
Epoch 9/11, Train Loss: 0.3589
Epoch 10/11, Train Loss: 0.3429
Epoch 11/11, Train Loss: 0.3324


In [17]:
path3 = '/kaggle/input/aksharantar/aksharantar_sampled/ben/ben_test.csv'
c, test_loader_ben, input_vocab, target_vocab, max_length, _ = load_data(path3, batch_size=64)  # Use correct path3

val_loss, val_accuracy = evaluate(trained_model, test_loader_ben, criterion, device)
print(f' Test Accuracy: {val_accuracy:.4f}')

 Test Accuracy: 30.0366


# **Prediction**

In [9]:
def decode_indices(indices, idx2token, target_vocab):
    valid_indices = []
    for idx in indices:
        if idx in idx2token and idx not in (target_vocab['<pad>'], target_vocab['<sos>'], target_vocab['<eos>']):
            valid_indices.append(idx)
#             print(valid_indices)
    decoded_text = ''
    for idx in valid_indices:
        decoded_text += idx2token[idx]
#         print(decoded_text)
    return decoded_text

In [10]:
def decode_indices_target(indices, idx2token, target_vocab):
    valid_indices = []
    for idx in indices:
        if idx in idx2token and idx not in (target_vocab['<pad>'], target_vocab['<sos>'], target_vocab['<eos>']):
            if idx < 10:
                valid_indices.append(idx)
            else:
                valid_indices.append(idx-3)
#             print(valid_indices)
    decoded_text = ''
    for idx in valid_indices:
        decoded_text += idx2token[idx]
#         print(decoded_text)
    return decoded_text

In [11]:
def pred(model, dataloader, device):
    model.eval()
    predictions = []
    actual = []
    with torch.no_grad():
        for latin, devanagari in dataloader:#, desc='Evaluating', unit='batch'):
            latin = latin.to(device)
            devanagari = devanagari.to(device)
            output = model(latin, devanagari,0)
            deb = devanagari.cpu().numpy()
            actual.append(deb)
            output = output.argmax(2)
            latin = latin.cpu().numpy()
            output = output.cpu().numpy()
            predictions.append((latin, output))
    return predictions, actual

path3 = '/kaggle/input/aksharantar/aksharantar_sampled/ben/ben_test.csv'
c, test_loader_ben, input_vocab, target_vocab, max_length, _ = load_data(path3, batch_size=64)  # Use correct path3

# Make sure to define the reverse dictionaries for converting indices back to text
latin_idx2token = {idx: char for char, idx in input_vocab.items()}
bangla_idx2token = {idx: char for char, idx in target_vocab.items()}

In [13]:
import csv
with open('seq2seq_results.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Input Text', 'Actual Target', 'Predicted Text'])
    writer.writerows(seq2seq_results)

In [14]:
df1 = pd.read_csv('seq2seq_results.csv')
df1

,Input Text,Actual Target,Predicted Text
0,kaarentabaahee,কারেন্টবাহী,কারেনতাহী
1,mashterpiece,মাস্টারপিস,মযাসটারিপে
2,cheeken,চিকেন,চিকেন
3,ekdaala,একডালা,একদালা
4,neerbachokra,নির্বাচকরা,নিরবচকরা
...,...,...,...
4090,samanjasyapurno,সামঞ্জস্যপূর্ণ,সমাজনযপূরণ
4091,fuds,ফুডস,ফুডস
4092,bannar,ব্যানার,বনননার
4093,songosthao,সংস্থাও,সংসথাও


In [12]:
test_predictions, actual = pred(model, test_loader_ben, device)
seq2seq_results = []
for (src_indices, output_indices),act_ind in zip(test_predictions,actual):
#     print(src_indices)
#     print('\njkl',output_indices)
    # Since our data loader might have batch size greater than 1, iterate through each example in the batch
    for i in range(src_indices.shape[0]):
        input_text = decode_indices(src_indices[i], latin_idx2token, input_vocab)
        actual_target_text = decode_indices(act_ind[i], bangla_idx2token, target_vocab)
        predicted_text = decode_indices_target(output_indices[i], bangla_idx2token, target_vocab)
        seq2seq_results.append([input_text, actual_target_text, predicted_text])
        print(f'SL. {i} Input Text: {input_text} -> Actual target: {actual_target_text} -> Predicted Text: {predicted_text}')

SL. 0 Input Text: kaarentabaahee -> Actual target: কারেন্টবাহী -> Predicted Text: কারেনতাহী
SL. 1 Input Text: mashterpiece -> Actual target: মাস্টারপিস -> Predicted Text: মযাসটারিপে
SL. 2 Input Text: cheeken -> Actual target: চিকেন -> Predicted Text: চিকেন
SL. 3 Input Text: ekdaala -> Actual target: একডালা -> Predicted Text: একদালা
SL. 4 Input Text: neerbachokra -> Actual target: নির্বাচকরা -> Predicted Text: নিরবচকরা
SL. 5 Input Text: neture -> Actual target: নেচার -> Predicted Text: নেতুরে
SL. 6 Input Text: michilkey -> Actual target: মিছিলকে -> Predicted Text: মিছিলকে
SL. 7 Input Text: chitfund -> Actual target: চিটফান্ড -> Predicted Text: চিটফুঁদ
SL. 8 Input Text: panchanan -> Actual target: পঞ্চানন -> Predicted Text: পঞচনন
SL. 9 Input Text: manna -> Actual target: মন্ন -> Predicted Text: মাননা
SL. 10 Input Text: portillo -> Actual target: পর্টিল্লো -> Predicted Text: পরতিলো
SL. 11 Input Text: quess -> Actual target: কুয়েস -> Predicted Text: কুয়েস
SL. 12 Input Text: budh -> Actua